In [1]:
# Import the functions from /tools
import math
import os

# You need to choose one of the api's or hf's "request_llm" function here. 
# from tools.hf_open_source_llm_request import request_llm
# from tools.api_request import request_gpt as request_llm
from tools.api_request import request_gemini as request_llm

from tools.db_detail import db_getdesc
from tools.sql_execute import sqlite_execute as execute
from tqdm import tqdm
import json
import argparse

def sql_evoke(query,db_name):
    result, execution_time ,executable = execute("datasets/cosql_dataset/database/"+db_name+"/"+db_name+".sqlite",query)
    return result 

def get_example(db_name):
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = sql_evoke(sql_query,db_name)
    column_example=""
    for table_name in result:
        column_example = column_example + table_name[0] + ":\n"
        sql_get_eg = "SELECT * FROM "+ table_name[0] +" LIMIT 3;"
        table_eg = sql_evoke(sql_get_eg,db_name)
        for table_data in table_eg:
            column_example = column_example + '('
            for column_data in table_data: 
                column_example = column_example + str(column_data) +','
            column_example = column_example[:-1] + ')\n'
    return column_example

def get_system(db_name): 
    # Get db schema prompt
    description = db_getdesc(db_name)
    column_example = get_example(db_name)
    question = "Database schema:\n" + description + "\nExamples for each table:"+ column_example + "\nBased on the provided information, if the user's question cannot be accurately answered with an SQL query, indicate whether the question is ambiguous(Problem is not enough to generate SQL with sure tables and columns) or unanswerable(Unable to answer questions based on database information) and explain to user why. If the question is answerable, output only SQL query without additional content. Answer normally if it's an everyday question"
    return question
    
print(db_getdesc("car_1"),get_example("car_1"))



continents(ContId:cont id type:number PRIMARY KEY|Continent:continent type:text|)
countries(CountryId:country id type:number PRIMARY KEY|CountryName:country name type:text|Continent:continent type:number|)
car_makers(Id:id type:number PRIMARY KEY|Maker:maker type:text|FullName:full name type:text|Country:country type:text|)
model_list(ModelId:model id type:number PRIMARY KEY|Maker:maker type:number|Model:model type:text|)
car_names(MakeId:make id type:number PRIMARY KEY|Model:model type:text|Make:make type:text|)
cars_data(Id:id type:number PRIMARY KEY|MPG:mpg type:text|Cylinders:cylinders type:number|Edispl:edispl type:number|Horsepower:horsepower type:text|Weight:weight type:number|Accelerate:accelerate type:number|Year:year type:number|)
 continents:
(1,america)
(2,europe)
(3,asia)
countries:
(1,usa,1)
(2,germany,2)
(3,france,2)
car_makers:
(1,amc,American Motor Company,1)
(2,volkswagen,Volkswagen,2)
(3,bmw,BMW,2)
model_list:
(1,1,amc)
(2,2,audi)
(3,3,bmw)
car_names:
(1,chevrolet,ch

In [2]:
from agents.selector import Selector
from agents.detector import Detector
from agents.decomposer import Decomposer
from agents.refiner import Refiner

# creat and name the agents
selector = Selector("Selector")
detector = Detector("Detector")
decomposer = Decomposer("Decomposer")
refiner = Refiner("Refiner")

# input_data = "hey!"

# output_selector = selector.process_input(input_data)
# output_detector = detector.process_input(input_data)
# output_decomposer = decomposer.process_input(input_data)
# output_refiner = refiner.process_input(input_data)

In [ ]:
def process_json_part(data, output_file):
    for index1,item in enumerate(tqdm(data)):
        # Initialnize messages
        # print("__________"+str(index1)+"___________")
        system_instruct = get_system(item['db_name'])
        messages = [{"role": "system", "content": system_instruct}]
        for index, turn in enumerate(item['turns']):
            if turn['isuser']:
                # update messages
                user_question = turn['text']
                print(str(index)+" type "+turn['type']+" Q: "+user_question)
                messages.append({"role": "user", "content": user_question})
                if index+1<len(item['turns']):
                    
                    print("________Select columns and values________")
                    
                    print("________Question type detect________")

                    print("________Decompose question and solve________")

                    print("________Refines erroneous SQL queries________")

                    
                    llm_response = request_llm(messages)
                    # llm record
                    print("\nLLM Response:")
                    print(llm_response)
                    item['turns'][index+1]['predict'] = llm_response
                    # update messages
                    g_ans = ""
                    if item['turns'][index+1]['text']:
                        g_ans = item['turns'][index+1]['text']
                    else:
                        g_ans = item['turns'][index+1]['query']
                    messages.append({"role": "assistant", "content": g_ans})
                    
        if not os.path.exists(output_file):
            with open(output_file, 'w') as f:
                items = [item]
                json.dump(items, f, indent=4)
                f.write('\n')
        else:
            with open(output_file, 'r') as f:
                try:
                    items = json.load(f)
                except json.JSONDecodeError:
                    print("\033[91mError:The file content is not in valid JSON format\033[0m")
                    # items = []

            if not isinstance(items, list):
                print("\033[91mError:The file content is not in valid JSON format\033[0m")
                # items = []

            items.append(item)

            with open(output_file, 'w') as f:
                json.dump(items, f, indent=4)
                f.write('\n') 

with  open('datasets/MMSQL_test.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    
process_json_part(data,'agenttest.json')